We verify the degree 7 construction computationally in 2 ways:
1) by emulating theoretical Gaussian cubature formulas within the construction,
2) by using exact Gaussian cubature formulas found in Stroud for ```dimension=2``` and ```dimension=3```.

We first create a lookup table that stores Eulerian idempotent values whenever they're computed. This prevents excessive re-computation of basis terms, since the Eulerian idempotent does not currently have an efficient algorithm (we use Lyndon basis identities for words of length $\leq 5$, but otherwise rely up the algorithm in ```free_lie_algebra.py``` written by Jeremy Reizenstein). Note: the first run of code will require a great deal of computation in high dimensions.

In [1]:
from cubature_construction import _eul, sym_prod

class basis():
    def __init__(self):
        """Helper class to act as a lookup table for Eulerian idempotent expansions. Whenever an expansion is called, it is indexed and saved for future lookup to prevent uncessessary computation."""
        self.basis_terms = {}
        self.eul_terms = {}

    def string(self,index):
        sorted_index = sorted(index, key=lambda x: (len(x), x))
        string = "("
        for i in sorted_index:
            string += "e(" + "".join([str(x) for x in i]) + "),"
        return string[:-1] + ")"

    def val_eul(self,index):
        """Computes or looks up value of Eulerian idempotent."""
        keys = self.eul_terms.keys()
        if str(index) in keys:
            return self.eul_terms[str(index)]
        else:
            value = _eul(index)
            self.eul_terms[str(index)] = value
            return value
    
    def val(self,index):
        """Computes or looks up value of symmetric product of Eulerian idempotents."""
        if isinstance(index, str):
            from re import findall
            pattern = r'e\((\d+)\)'
            matches = findall(pattern, index)
            index =  [[int(digit) for digit in num] for num in matches]
        keys = self.basis_terms.keys() 
        if str(index) in keys:
            return self.basis_terms[str(index)]
        else:
            value = sym_prod([self.val_eul(x) for x in index])
            self.basis_terms[str(index)] = value
            return value

In [2]:
b = basis() # initialise lookup table

For approach 1., we developed series of classes stored in ```exponentiator.py``` that emulate the mathematical objects required for the construction. That is, Gaussian cubature formulas, Eulerian idempotent terms, linear combinations of these terms and exponentiation in the symmetrised Eulerian idempotent basis. First, however, we can write the LHS of [LV04],
$$
    \mathbb{E}\Big [\varepsilon_0+\sum_{i=1}^d \varepsilon_i\otimes\varepsilon_i\Big ],
$$
in the Eulerian idempotent basis. We add the option to include or exclude the drift component, $\varepsilon_0$.

In [3]:
from collections import Counter

def eulerian_lhs(dimension : int, drift : bool = True):
    """Returns (emulated) degree 7 LHS expansion from Lyons-Victoir written in the symmetrised Eulerian idomptent (redundant) basis."""
    expansion = Counter({"1" : 1.})
    for i in range(1,dimension+1):
        expansion += Counter({b.string([[i],[i]]) : 1/2})
        for j in range(1,dimension+1):
            expansion += Counter({
                b.string([[i,j],[i,j]]) : 1/4,
                b.string([[i],[i,j,j]]) : 1/2,
                b.string([[i],[i],[j],[j]]) : 1/8
            })
            for k in range(1,dimension+1):
                expansion += Counter({b.string([[i],[i,j,j,k,k]]) : 1/24})
                expansion += Counter({b.string([[j],[i,i,j,k,k]]) : 1/24})
                expansion += Counter({b.string([[k],[i,i,j,j,k]]) : 1/24})
                expansion += Counter({b.string([[i,j],[i,j,k,k]]) : 1/12})
                expansion += Counter({b.string([[i,k],[i,j,j,k]]) : 1/12})
                expansion += Counter({b.string([[j,k],[i,i,j,k]]) : 1/12})
                expansion += Counter({b.string([[i,j,k],[i,j,k]]) : 1/12})
                
                expansion += Counter({b.string([[i,j,j],[i,k,k]]) : 1/8})
                expansion += Counter({b.string([[i],[i],[j],[j,k,k]]) : 1/4})
                expansion += Counter({b.string([[i],[i],[j,k],[j,k]]) : 1/8})
                expansion += Counter({b.string([[i],[j],[i,k],[j,k]]) : 1/6})
                expansion += Counter({b.string([[i],[i],[j],[j],[k],[k]]) : 1/48})
        if drift:
            expansion += Counter({b.string([[0]]) : 1})
            expansion += Counter({b.string([[0],[0]]) : 1/2})
            expansion += Counter({b.string([[0],[0],[0]]) : 1/6})
            for i in range(1,dimension+1):
                expansion += Counter({b.string([[0,i,i]]) : 1/2})
                expansion += Counter({b.string([[0],[i],[i]]) : 1/2})
                expansion += Counter({b.string([[0],[0,i,i]]) : 1/2})
                expansion += Counter({b.string([[0,i],[0,i]]) : 1/6})
                expansion += Counter({b.string([[0],[0],[i],[i]]) : 1/4})
                for j in range(1,dimension+1):
                    expansion += Counter({b.string([[0,i,i,j,j]]) : 1/24})
                    expansion += Counter({b.string([[i,i,0,j,j]]) : 1/24})
                    expansion += Counter({b.string([[i,i,j,j,0]]) : 1/24})
                    expansion += Counter({b.string([[0],[i],[i,j,j]]) : 1/2})
                    expansion += Counter({b.string([[j],[j],[0,i,i]]) : 1/4})
                    expansion += Counter({b.string([[0],[i,j],[i,j]]) : 1/4})
                    expansion += Counter({b.string([[i],[0,j],[i,j]]) : 1/3})
                    expansion += Counter({b.string([[0],[i],[i],[j],[j]]) : 1/12})
            
    return dict(expansion)

We now write our proposed degree 7 cubature formula using the emulated mathematical objects in ```exponentiator.py```.

In [4]:
from exponentiator import gaussian_cubature,bernoulli_cubature,basis_term,coefficient,TupleCounter,numeric
import numpy as np

def _add_tuple(lst, new_tuple):
    """Helper function to combine tuples with repeated keys when appending to a list."""
    key, value = new_tuple
    tuple_dict = dict(lst)
    if key in tuple_dict:
        tuple_dict[key] = coefficient(tuple_dict[key].terms + value.terms)
    else:
        tuple_dict[key] = value
    return list(tuple_dict.items())

def eulerian_wiener_cubature(dimension, drift : bool = True):
    """Returns (emulated) degree 7 Wiener space cubature construction in the Eulerian idomptotent basis."""
    degree = 7

    gauss_i = [gaussian_cubature(degree) for i in range(dimension)]
    ber_i = [bernoulli_cubature(degree) for i in range(dimension*(dimension + 1))]
    ber = bernoulli_cubature(degree)

    structure = []
    for i in range(1,dimension+1):
        structure.append((basis_term(str(i)),coefficient([{gauss_i[i-1] : (1,)}])))
        for j in range(1,dimension+1):
            tc1 = TupleCounter()
            tc1.update([(numeric(np.sqrt(1/3)),(1,)),(gauss_i[i-1],(1,)),(ber_i[j-1],(1,))])
            tc2 = TupleCounter()
            tc2.update([(numeric(np.sqrt(1/6)),(1,)),(ber_i[i*dimension + (j-1)],(1,))])
            c = coefficient([dict(tc1.counter),dict(tc2.counter)])
            structure.append((basis_term(str(i)+str(j)), c))
            structure.append((basis_term(str(i)+str(j)+str(j)), coefficient([{numeric(1/2) : (1,), gauss_i[i-1] : (1,)}])))
            
            for k in range(1,dimension+1):
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(j)+str(k)),
                    coefficient([{numeric(1/np.sqrt(6)) : (1,), ber_i[i*dimension + (j-1)] : (1,), gauss_i[k-1] : (1,), ber : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(j)+str(k)+str(k)),
                    coefficient([{numeric((1/2)*np.sqrt(1/6)) : (1,), ber_i[i*dimension + (j-1)] : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(j)+str(j)+str(k)),
                    coefficient([{numeric((1/2)*np.sqrt(1/6)) : (1,), ber_i[i*dimension + (k-1)] : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(i)+str(j)+str(k)),
                    coefficient([{numeric((1/2)*np.sqrt(1/6)) : (1,), ber_i[j*dimension + (k-1)] : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(i)+str(j)+str(j)+str(k)),
                    coefficient([{numeric(1/24) : (1,), gauss_i[k-1] : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(i)+str(j)+str(k)+str(k)),
                    coefficient([{numeric(1/24) : (1,), gauss_i[j-1] : (1,)}])
                ))
                structure = _add_tuple(structure, (
                    basis_term(str(i)+str(j)+str(j)+str(k)+str(k)),
                    coefficient([{numeric(1/24) : (1,), gauss_i[i-1] : (1,)}])
                ))
    if drift:
        structure.append((basis_term(str(0)),coefficient([{}])))
        for i in range(1,dimension+1):
            c = coefficient([{numeric(np.sqrt(1/3)) : (1,), ber_i[i-1] : (1,)}])
            structure.append((basis_term("0"+str(i)), c))
            structure.append((basis_term("0"+str(i)+str(i)), coefficient([{numeric(1/2) : (1,)}])))
            for j in range(1,dimension+1):
                structure = _add_tuple(structure, (
                            basis_term(str(i)+str(i)+str(j)+str(j)+"0"),
                            coefficient([{numeric(1/24) : (1,)}])
                        ))
                structure = _add_tuple(structure, (
                            basis_term(str(i)+str(i)+"0"+str(j)+str(j)),
                            coefficient([{numeric(1/24) : (1,)}])
                        ))
                structure = _add_tuple(structure, (
                        basis_term("0"+str(i)+str(i)+str(j)+str(j)),
                        coefficient([{numeric(1/24) : (1,)}])
                    ))
    return structure

The evaluation of the RHS in [LV04],
$$
\mathbb{E}\Big [\sum_{i=1}^n\lambda_i\exp \mathcal{L}_i]
$$
using the cubature formula can now be computed. Below is an example for ```dimension=3```. Computing higher dimensions becomes highly computation.

In [5]:
from exponentiator import exponentiate

degree = 7
dimension = 3
drift = True
e = {}
for elm in exponentiate(eulerian_wiener_cubature(dimension,drift),degree):
    e.update(elm)
e

{'1': 1.0,
 '(e(0))': 1.0,
 '(e(011))': 0.5,
 '(e(11110))': 0.041666666666666664,
 '(e(11011))': 0.041666666666666664,
 '(e(01111))': 0.041666666666666664,
 '(e(11220))': 0.041666666666666664,
 '(e(11022))': 0.041666666666666664,
 '(e(01122))': 0.041666666666666664,
 '(e(11330))': 0.041666666666666664,
 '(e(11033))': 0.041666666666666664,
 '(e(01133))': 0.041666666666666664,
 '(e(022))': 0.5,
 '(e(22110))': 0.041666666666666664,
 '(e(22011))': 0.041666666666666664,
 '(e(02211))': 0.041666666666666664,
 '(e(22220))': 0.041666666666666664,
 '(e(22022))': 0.041666666666666664,
 '(e(02222))': 0.041666666666666664,
 '(e(22330))': 0.041666666666666664,
 '(e(22033))': 0.041666666666666664,
 '(e(02233))': 0.041666666666666664,
 '(e(033))': 0.5,
 '(e(33110))': 0.041666666666666664,
 '(e(33011))': 0.041666666666666664,
 '(e(03311))': 0.041666666666666664,
 '(e(33220))': 0.041666666666666664,
 '(e(33022))': 0.041666666666666664,
 '(e(03322))': 0.041666666666666664,
 '(e(33330))': 0.04166666666666

In [6]:
eulerian_lhs(dimension, drift)

{'1': 1.0,
 '(e(1),e(1))': 0.5,
 '(e(11),e(11))': 0.25,
 '(e(1),e(111))': 0.5,
 '(e(1),e(1),e(1),e(1))': 0.125,
 '(e(1),e(11111))': 0.125,
 '(e(11),e(1111))': 0.25,
 '(e(111),e(111))': 0.20833333333333331,
 '(e(1),e(1),e(1),e(111))': 0.25,
 '(e(1),e(1),e(11),e(11))': 0.29166666666666663,
 '(e(1),e(1),e(1),e(1),e(1),e(1))': 0.020833333333333332,
 '(e(1),e(11122))': 0.08333333333333333,
 '(e(2),e(11112))': 0.041666666666666664,
 '(e(11),e(1122))': 0.08333333333333333,
 '(e(12),e(1112))': 0.16666666666666666,
 '(e(112),e(112))': 0.08333333333333333,
 '(e(111),e(122))': 0.25,
 '(e(1),e(1),e(1),e(122))': 0.25,
 '(e(1),e(1),e(12),e(12))': 0.29166666666666663,
 '(e(1),e(1),e(1),e(1),e(2),e(2))': 0.0625,
 '(e(1),e(11133))': 0.08333333333333333,
 '(e(3),e(11113))': 0.041666666666666664,
 '(e(11),e(1133))': 0.08333333333333333,
 '(e(13),e(1113))': 0.16666666666666666,
 '(e(113),e(113))': 0.08333333333333333,
 '(e(111),e(133))': 0.25,
 '(e(1),e(1),e(1),e(133))': 0.25,
 '(e(1),e(1),e(13),e(13))': 

We can verify the construction by comparing the LHS expansion written earlier. Equally, we can evaluate the Lie elements in the tensor algebra and compare with the LHS in the tensor algebra (implemented by ```verify_lhs(...)``` from ```cubature_construction.py```), which is performed here.

In [7]:
from cubature_construction import verify_lhs
from free_lie_algebra import Word, Elt, distance

rhs = Elt([{Word([]):1}])
for x in e.items():
    if x[0] != "1":
        rhs += x[1]*b.val(x[0])

distance(rhs, verify_lhs(degree,dimension,drift))

6.66529543960635e-17

For approach 2., we use ```wiener_cubature(...)``` from ```cubature_construction.py``` which implements the proposed cubature formula directly in the free Lie algebra as ```Elt``` objects from Jeremy Reizenstein's ```free_lie_algebra.py```.

In [8]:
from cubature_construction import wiener_cubature

degree = 7
dimension = 3
drift = True

lie_poly, lam = wiener_cubature(degree, dimension, drift)

We demonstrate that the $L_1$ distance between the exponentiated cubature formula, ```verify_rhs(...)```, and the expected signature for Brownian motion, ```verify_lhs(...)```, is (effectively) zero. 

In [9]:
from cubature_construction import verify_rhs

rhs = verify_rhs(degree, dimension, lie_poly, lam)
lhs = verify_lhs(degree, dimension, drift)

distance(lhs,rhs)

1.4234785643718485e-13